# 월간 Melon Top100 크롤링

In [53]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import random

# 크롤링
* 2022년 1 ~ 3월

## 멜론 사이트 접속

In [54]:
driver = webdriver.Chrome('../chromedriver')
url = 'https://www.melon.com/chart/index.htm'
driver.get(url)
driver.maximize_window()

<ipython-input-54-40ce6c51e4f5>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver')


In [55]:
# 차트 파인더로 이동
chartfinder_button = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button')
chartfinder_button.click()
time.sleep( random.uniform(1, 2) )

# 주간차트 선택
monthchart_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a')
monthchart_button.click()
time.sleep( random.uniform(1, 2) )

# 연대선택(2020년대)
selecttopyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label')
selecttopyear_button.click()
time.sleep( random.uniform(1, 2) )

# 연도선택(2022년)
selectyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label')
selectyear_button.click()
time.sleep( random.uniform(1, 2) )

# 월간 선택(1월)
selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label')
selectmonth.click()
time.sleep( random.uniform(1, 2) )

# 장르 선택
selectgenre_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label')
selectgenre_button.click()
time.sleep( random.uniform(1, 2) )

# 검색 버튼 클릭
search_button = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span')
search_button.click()
time.sleep( random.uniform(1, 2) )

# 처음 검색 한번 한뒤에 진행해야 정상적으로 첫번째 정보 정상적으로 수집.
# 바로 진행하면 2013년 자료 수집이 되는 문제 발생.

<ipython-input-55-ea67c79b0967>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  chartfinder_button = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button')
<ipython-input-55-ea67c79b0967>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  monthchart_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a')
<ipython-input-55-ea67c79b0967>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selecttopyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label')
<ipython-input-55-ea67c79b0967>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/d

## 월간 크롤링 함수

In [56]:
def month_crawl(month):
    # 월간 선택
    m = month
    selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{m}]/span/label')
    selectmonth.click()
    time.sleep( random.uniform(1, 2) )

    # 장르 선택
    selectgenre_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label')
    selectgenre_button.click()
    time.sleep( random.uniform(1, 2) )

    # 검색 버튼 클릭
    search_button = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span')
    search_button.click()
    time.sleep( random.uniform(1, 2) )
    
    # 정보 수프에 담기
    soup = bs(driver.page_source, 'lxml')

    # 검색 제목 수집
    search_title = soup.find('div', 'serch_cnt')
    title = search_title.text[:-8].replace(' ', '')

    # 순위 날짜
    rank_date = title.replace('년', '-')
    rank_date = rank_date.replace('월', '-')
    rank_date = rank_date.replace(' ', '') + '01'
    
    repeat_rank_date_list = []
    for i in range(100):
        repeat_rank_date_list.append(rank_date)

    # 순위 수집
    rank_list = []
    for i in range(1, 101):
        rank_list.append(i)

    # 곡정보
    song_list = []
    song_info = soup.find_all('div', 'wrap_song_info')
    for one in song_info:
        temp = one.find('div', 'ellipsis rank01').text.strip()
        if '19금\n' in temp:
            temp = temp.replace('19금\n', '')
        song_list.append(temp)

    # 가수
    artist_list = []
    for one in song_info:
        temp = one.find('span', 'checkEllipsis').text
        artist_list.append(temp)

    # 앨범
    album_list = []
    for one in song_info:
        temp = one.find('div', 'ellipsis rank03').text.strip()
        album_list.append(temp)

    # 수집 데이터 확인
    print('수집한 기간 : ', title)
    print('수집한 순위 날짜 개수 : ', len(repeat_rank_date_list))
    print('수집한 순위 개수 : ', len(rank_list))
    print('수집한 곡 개수 : ', len(song_list))
    print('수집한 가수 개수 : ', len(artist_list))
    print('수집한 앨범 개수 : ', len(album_list))

    # 데이터 셋 만들기
    dict = {'날짜':repeat_rank_date_list,'순위':rank_list, '곡':song_list, '가수':artist_list, '앨범':album_list}
    df = pd.DataFrame(dict)

    # 파일로 출력
    date = rank_date.replace('-','')
    file_name = f'month_melon_{date}.xlsx'
    path = '../crawling_data/month_melon/'
    df.to_excel(path + file_name, index=False)

    print(f'{title}')
    print(f"{file_name} 파일 생성 완료")

In [57]:
for i in range(1, 4):
    month_crawl(i)
    print()

<ipython-input-56-ffcb57c4d813>:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{m}]/span/label')
<ipython-input-56-ffcb57c4d813>:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectgenre_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label')
<ipython-input-56-ffcb57c4d813>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_button = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span')


수집한 기간 :  2022년01월
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
2022년01월
month_melon_20220101.xlsx 파일 생성 완료

수집한 기간 :  2022년02월
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
2022년02월
month_melon_20220201.xlsx 파일 생성 완료

수집한 기간 :  2022년03월
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
2022년03월
month_melon_20220301.xlsx 파일 생성 완료



# 코드 마지막